In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.4/590.4 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires ml-dtypes~=0.2

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-02-02 11:41:57--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2024-02-02 11:41:57 (11.2 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-02-02 11:41:57--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [3]:
train_data = pd.read_table(train_file_path , names = ['label' , 'message'] , header = 0)
print(train_data)

test_data = pd.read_table(test_file_path , names = ['label' , 'message'] , header = 0)
print(test_data)

     label                                            message
0      ham                           you can never do nothing
1      ham  now u sound like manky scouse boy steve,like! ...
2      ham  mum say we wan to go then go... then she can s...
3      ham  never y lei... i v lazy... got wat? dat day ü ...
4      ham  in xam hall boy asked girl tell me the startin...
...    ...                                                ...
4173   ham  just woke up. yeesh its late. but i didn't fal...
4174   ham  what do u reckon as need 2 arrange transport i...
4175  spam  free entry into our £250 weekly competition ju...
4176  spam  -pls stop bootydelious (32/f) is inviting you ...
4177   ham  tell my  bad character which u dnt lik in me. ...

[4178 rows x 2 columns]
     label                                            message
0      ham         not much, just some textin'. how bout you?
1      ham  i probably won't eat at all today. i think i'm...
2      ham  don‘t give a flying monkeys wot t

In [4]:
!pip install indexer
!pip install pyspellchecker
import re
import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords' , quiet=True)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from spellchecker import SpellChecker
import time
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)


lematizer = WordNetLemmatizer()
spelling = SpellChecker()
#function to clean and correct comments
def preprocess(comment):
    comment = re.sub(r'[^A-Za-z0-9\s]', '', comment)

    cspellwords = [spelling.correction(word) if spelling.correction(word) is not None else word for word in comment.split()]
    token = word_tokenize(' '.join(cspellwords))

    stop_words = set(stopwords.words('english'))
    normalized_words = [word.lower() for word in token if word.lower() not in stop_words]

    lemmatized_words = []
    for word, tag in nltk.pos_tag(normalized_words):
        pos = tag[0].lower()  # Convert POS tag to WordNet format
        pos = pos if pos in ['a', 'r', 'n', 'v'] else 'n'

        lemma = lematizer.lemmatize(word, pos=pos)
        lemmatized_words.append(lemma)

    print(lemmatized_words)
    return ' '.join(lemmatized_words)

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 19.7 MB/s eta 0:00:00


In [5]:
# Merge multiple lines within comments into a single line
train_data['message'] = train_data['message'].str.replace('\n', ' ')
train_data['message'] = train_data['message'].apply(preprocess)

test_data['message'] = test_data['message'].str.replace('\n', ' ')
test_data['message'] = test_data['message'].apply(preprocess)

Streaming output truncated to the last 5000 lines.
['cop', 'lar', '1', 'u', 'never', 'ask', 'go', 'ah', 'say', 'u', 'would', 'ask', 'fry', 'say', 'u', 'ask', 'today']
['urgent', 'attempt', 'contact', 'u', '900', 'prize', 'yesterday', 'still', 'await', 'collection', 'claim', 'call', '09061702893']
['send', 'id', 'password']
['free']
['die', 'want', 'u', 'stuff']
['wish', 'family', 'merry', 'x', 'ma', 'happy', 'new', 'year', 'advance']
['sorry', 'cant', 'help']
['well', 'nice', 'bad', 'cant', 'eat']
["'ve", 'already', 'get', 'flaky', 'parent', "n't", 'suppose', 'child', 'job', 'support', 'parentnot', 'ride', 'age', 'anyway', 'suppose', 'support', 'give', 'hurt', 'unintentional', 'hurt', 'nonetheless']
['cheer', 'yeah', 'goodie', 'shame', 'u', 'nova', 'come', 'c', 'ya', 'gaily']
['hi', 'pique', 'meet', 'u', '11', 'tar']
['today', 'song', 'dedicate', 'day', 'song', 'u', 'dedicate', 'send', 'valuable', 'find', 'first', 'rely']
['reply', 'win', '100', 'weekly', 'professional', 'sport', 'tige

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from keras.preprocessing.sequence import pad_sequences

tfidf = TfidfVectorizer(max_features=5000)
X_train = tfidf.fit_transform(train_data['message']).toarray()
X_test = tfidf.transform(test_data['message']).toarray()

# Encoding the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(train_data['label'])
y_test_encoded = label_encoder.transform(test_data['label'])

# Reshape the input data for LSTM
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test= X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train_encoded, epochs=10, batch_size=64, validation_data=(X_test, y_test_encoded), verbose=2)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
66/66 - 2s - 24ms/step - accuracy: 0.8626 - loss: 0.5864 - val_accuracy: 0.8735 - val_loss: 0.4452
Epoch 2/10
66/66 - 1s - 15ms/step - accuracy: 0.8836 - loss: 0.3293 - val_accuracy: 0.9109 - val_loss: 0.2398
Epoch 3/10
66/66 - 0s - 6ms/step - accuracy: 0.9318 - loss: 0.1888 - val_accuracy: 0.9469 - val_loss: 0.1548
Epoch 4/10
66/66 - 0s - 6ms/step - accuracy: 0.9633 - loss: 0.1222 - val_accuracy: 0.9640 - val_loss: 0.1101
Epoch 5/10
66/66 - 1s - 9ms/step - accuracy: 0.9773 - loss: 0.0819 - val_accuracy: 0.9780 - val_loss: 0.0865
Epoch 6/10
66/66 - 1s - 9ms/step - accuracy: 0.9853 - loss: 0.0595 - val_accuracy: 0.9822 - val_loss: 0.0737
Epoch 7/10
66/66 - 0s - 6ms/step - accuracy: 0.9896 - loss: 0.0450 - val_accuracy: 0.9837 - val_loss: 0.0659
Epoch 8/10
66/66 - 0s - 5ms/step - accuracy: 0.9910 - loss: 0.0350 - val_accuracy: 0.9837 - val_loss: 0.0617
Epoch 9/10
66/66 - 0s - 6ms/step - accuracy: 0.9934 - loss: 0.0280 - val_accuracy: 0.9851 - val_loss: 0.0588
Epoch 10/10
66/66

In [15]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # Preprocess the input text
    preprocessed_text = preprocess(pred_text)

    # Vectorize the preprocessed text using TF-IDF
    text = tfidf.transform([preprocessed_text]).toarray()

    # Reshape the input data for LSTM
    text = text.reshape(text.shape[0], 1, text.shape[1])

    # Make prediction using the trained model
    prediction = model.predict(text)[0][0]

    # Decode the prediction label
    label = 'ham' if prediction < 0.5 else 'spam'

    return [prediction, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

['today']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
[0.018881472, 'ham']


In [16]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


['today']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
['sale', 'today', 'stop', 'text', 'call', '98912460324']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
["n't", 'want', 'go', 'try', 'different', 'day', 'available', 'sat']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
['new', 'mobile', 'video', 'service', 'live', 'install', 'phone', 'start', 'watch']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
['1000', 'cash', 'call', 'claim', 'prize']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
['ill', 'bring', 'tomorrow', "n't", 'forget', 'milk']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
['wow', 'arm', 'alright', 'happen', 'one', 'time']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
You passed the challenge. Great job!
